In [ ]:
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import torch
import json
import re
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
with open("ww_data.json") as f: data = json.load(f)

In [4]:
app_date = datetime.datetime(2024, 9, 15)

In [5]:
jobs = []
divisions = []
for key in data:
    if (
        key.startswith("coopJob_") and 
        app_date < datetime.datetime.strptime(re.sub("\s+", " ", data[key]["pageData"]["Application Information"]["Application Deadline"]), "%B %d, %Y %I:%M %p")
    ): jobs.append(data[key])
    elif key.startswith("division_"): divisions.append(data[key])

In [6]:
pd.DataFrame(data={"col1": [{1:2}, {2:3}, {3:4}], "col2": [4, 5, 6]}).iloc

col1    object
col2     int64
dtype: object

: 

In [15]:
columns = {
    "Is For My Program": pd.Series(dtype="bool"), 
    "Job Title": pd.Series(dtype="str"), 
    "Job Summary": pd.Series(dtype="str"), 
    "Job Responsibilities": pd.Series(dtype="str"), 
    "Required Skills": pd.Series(dtype="str"), 
    "Min Salary": pd.Series(dtype="float"), 
    "Max Salary": pd.Series(dtype="float"), 
    "Location": pd.Series(dtype="str"), 
    "Work Arrangement": pd.Series(dtype="str"), 
    "Duration": pd.Series(dtype="int"), 
    "Applications": pd.Series(dtype="int"), 
    "Hiring History": pd.Series(dtype="list[int]"), 
    "Hires by Faculty": pd.Series(dtype="dict[str, int]"), 
    "Hires by Work Term Number": pd.Series(dtype="dict[int, int]"), 
    "Most Frequently Hired Programs": pd.Series(dtype="dict[str, int]"), 
    "Work Term Ratings": pd.Series(dtype="int"), 
    "Availability of employer support": pd.Series(dtype="float"), 
    "Opportunities to learn or develop new skills": pd.Series(dtype="float"), 
    "Opportunities to make meaningful contributions at work": pd.Series(dtype="float"), 
    "Opportunities to expand your professional network": pd.Series(dtype="float"), 
    "Appropriate compensation and/or benefits": pd.Series(dtype="float"), 
    "How closely your work was related to your academic program": pd.Series(dtype="float"), 
    "How closely your work was related to the skills you are developing at university": pd.Series(dtype="float")
}
jobs_df = pd.DataFrame(data=[], columns=columns.keys())

TypeError: data type 'list[int]' not understood

In [52]:
job_descriptions = [
    re.sub(
        r"\n\n+",
        "\n",
        BeautifulSoup(
            f"\
Job Title: {job['pageData']['Job Posting Information']['Job Title']}\n\
Job Summary: {job['pageData']['Job Posting Information']['Job Summary']}\n\
Job Responsibilities: {job['pageData']['Job Posting Information']['Job Responsibilities']}\n\
Required Skills: {job['pageData']['Job Posting Information']['Required Skills']}",
            "html.parser"
        ).get_text()
    )
    for job in jobs
]

In [93]:
model = SentenceTransformer('nomic-ai/nomic-embed-text-v1', trust_remote_code=True).to(device)

C:\Users\parth\.cache\huggingface\modules\transformers_modules\nomic-ai\nomic-bert-2048\4bb68f63016e88e53e48df904c6ab4e6f718e198\modeling_hf_nomic_bert.py:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related 